In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import soundfile as sf
from pathlib import Path
import re
import json
import os
import sys
import glob
print(os.getcwd())
sys.path.append('..\..\soundbay')
from utils.metadata_processing import (reorder_columns_to_default_view, 
                                       correct_call_times_with_duration,
                                       non_overlap_df, bg_from_non_overlap_calls)

C:\Users\amitg\Documents\GitHub\soundbay\datasets\Manatees


In [13]:
sys.path.append('C:/Users')
selection_path = Path('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/BB_analysis/2017-Jul-08-12-67407878-RestingHole1A')
#recording_path = Path('/mnt/c/users/amitg/Documents/Deep Voice/HF WAV Manatee Samples/170708155527')
selection_path2 = Path('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/BB_analysis/2017-Jul-14-17-67407878-RestingHole1A')
# selection_path = Path('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/verified_annotations')
# predictions_path = Path('C:/Users/amitg/Documents/Deep_Voice/HF_WAV_Manatee_Samples/Placencia/placencia_raven_001')

filelist = list(selection_path.glob("*.txt"))
filelist2 = list(selection_path2.glob('*.txt'))
# filelist_pred = list(predictions_path.glob("*.csv"))

filelist_short = []
for file in filelist:
    print(file)
    name_ext = os.path.basename(file)
    name, txt = os.path.splitext(name_ext)
    name = re.search(r'67\d{6,6}.\d{12,12}', file.as_posix()).group() 
    filelist_short.append(name)
filelist_short2 = []
for file in filelist2:
    name_ext = os.path.basename(file)
    name, txt = os.path.splitext(name_ext)
    name = re.search(r'67\d{6,6}.\d{12,12}', file.as_posix()).group() 
    filelist_short2.append(name)

# filelist_pred_short = []
# for file in filelist_pred:
#     name_ext = os.path.basename(file)
#     name, txt = os.path.splitext(name_ext)
#     name = re.search(r'67\d{6,6}.\d{12,12}', file.as_posix()).group() 
#     filelist_pred_short.append(name)

C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\BB_analysis\2017-Jul-08-12-67407878-RestingHole1A\67407878.170708215525_BB.txt
C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\BB_analysis\2017-Jul-08-12-67407878-RestingHole1A\67407878.170708232525_BB.txt
C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\BB_analysis\2017-Jul-08-12-67407878-RestingHole1A\67407878.170709035524_BB.txt
C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\BB_analysis\2017-Jul-08-12-67407878-RestingHole1A\67407878.170709082523_BB.txt
C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\BB_analysis\2017-Jul-08-12-67407878-RestingHole1A\67407878.170709095523_BB.txt
C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\BB_analysis\2017-Jul-08-12-67407878-RestingHole1A\67407878.170709112522_BB.txt
C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\BB_analysis\2017-Jul-08-12-67407878-RestingHole1A\67407878.170709232519_BB.txt
C:\Users\amitg\Documents\Deep_Voic

In [14]:
# filelist_pred_short
filelist_short

['67407878.170708215525',
 '67407878.170708232525',
 '67407878.170709035524',
 '67407878.170709082523',
 '67407878.170709095523',
 '67407878.170709112522',
 '67407878.170709232519',
 '67407878.170710005518',
 '67407878.170710052517',
 '67407878.170710095516']

In [17]:
# textfile = open("ManateesList2.txt","w")
# for f in filelist_short:
#     textfile.write(f+"\n")
# textfile.close()

In [15]:
metadata=[]
metadata2 = []
for file in filelist:
    dfTemp = pd.read_csv(file, sep="\t")
    dfTemp['filename'] = re.search(r'20\d{10,10}', file.as_posix()).group() 
    dfTemp['StartMicInWater'] =np.amin(dfTemp['begin_time'])
    dfTemp['EndMicInWater'] =np.amax(dfTemp['end_time'])
    dfTemp['filepath'] = os.path.basename(selection_path)#os.path.join(,file,'.txt')
    metadata.append(dfTemp)
for file in filelist2:
    dfTemp = pd.read_csv(file, sep="\t")
    dfTemp['filename'] = re.search(r'17\d{10,10}', file.as_posix()).group() 
    dfTemp['StartMicInWater'] =np.amin(dfTemp['Begin Time (s)'])
    dfTemp['EndMicInWater'] =np.amax(dfTemp['End Time (s)'])
    dfTemp['filepath'] = os.path.basename(selection_path2)#os.path.join(,file,'.txt')
    metadata2.append(dfTemp)
# metadata_pred=[]
# for file in filelist_pred:
#     dfTemp = pd.read_csv(file, sep="\t")
#     dfTemp['filename'] = re.search(r'(20|21)\d{10}', file.as_posix()).group() 
#     dfTemp['StartMicInWater'] =np.amin(dfTemp['Begin Time (s)'])
#     dfTemp['EndMicInWater'] =np.amax(dfTemp['End Time (s)'])
#     dfTemp['filepath'] = os.path.basename(selection_path)#os.path.join(,file,'.txt')
#     metadata_pred.append(dfTemp)

metadata = pd.concat(metadata)
# metadata.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True) # already done
metadata2 = pd.concat(metadata2)
# metadata2.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True)
# metadata_pred = pd.concat(metadata_pred)
# metadata_pred.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True) # already done
frames = [metadata,metadata2]
PositiveCalls = pd.concat(frames, axis=0)
# PositiveCalls = metadata
PositiveCalls = PositiveCalls.drop(['Type of NLP','UpdatedNLP','SNR code','manatee','Manatee', 'MANATEE VOC',
                                    'mANATEE','2ndMeasure','AB Notes','AB notes'], axis=1)
PositiveCalls['label'] = np.ones((len(PositiveCalls),), dtype=int) #assuming everything is calls. TODO: filter out by notes
metadata_pred['label'] = np.ones((len(metadata_pred),), dtype=int)


AttributeError: 'NoneType' object has no attribute 'group'

In [38]:
PositiveCalls
# metadata_pred

,Selection,View,Channel,begin_time,end_time,Low Freq (Hz),Delta Time (s),Delta Freq (Hz),High Freq (Hz),filename,StartMicInWater,EndMicInWater,filepath,label
0,1,Spectrogram 1,1,412.844775,413.186087,9144.000,0.3413,19682.700,28826.700,201212101523,412.844775,10102.45615,verified_annotations,1
1,2,Spectrogram 1,1,415.558280,415.731153,3719.600,0.1729,15498.200,19217.800,201212101523,412.844775,10102.45615,verified_annotations,1
2,3,Spectrogram 1,1,418.951454,419.321578,5424.400,0.3701,19527.800,24952.200,201212101523,412.844775,10102.45615,verified_annotations,1
3,4,Spectrogram 1,1,463.214077,463.542092,0.000,0.3280,12553.600,12553.600,201212101523,412.844775,10102.45615,verified_annotations,1
4,5,Spectrogram 1,1,496.355213,496.571045,7749.100,0.2158,15464.015,23213.115,201212101523,412.844775,10102.45615,verified_annotations,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,606,Spectrogram 1,1,5390.534050,5390.843350,2943.332,0.3093,11773.328,14716.660,201212221503,12.793650,5656.32313,verified_annotations,1
105,607,Spectrogram 1,1,5533.975080,5534.332820,6204.862,0.3577,3181.981,9386.843,201212221503,12.793650,5656.32313,verified_annotations,1
106,608,Spectrogram 1,1,5522.553430,5522.911170,6125.312,0.3577,7795.853,13921.165,201212221503,12.793650,5656.32313,verified_annotations,1
107,609,Spectrogram 1,1,5595.790620,5595.947130,16228.101,0.1565,1431.891,17659.992,201212221503,12.793650,5656.32313,verified_annotations,1


In [50]:
PositiveCalls = non_overlap_df(PositiveCalls)
annotation_file = bg_from_non_overlap_calls(PositiveCalls)
annotation_file.to_csv('Placencia_annotations_bg_from_calls.csv', index=False)

In [39]:
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

def calculate_overlap(segment1, segment2):
    # Calculate overlap duration between two segments
    overlap_duration = max(0, min(segment1["end_time"], segment2["end_time"]) - max(segment1["begin_time"], segment2["begin_time"]))
    return overlap_duration

def evaluate_segments(ground_truth_df, predictions_df):
    # Initialize counters
    TP, FP, FN = 0, 0, 0

    # Iterate through predicted segments
    for _, pred_row in predictions_df.iterrows():
        if pred_row["label"] == 1:
            # Check for overlap with ground truth positive segments
            overlaps = ground_truth_df.apply(lambda row: calculate_overlap(pred_row, row), axis=1)
            if overlaps.max() > 0:
                TP += 1
            else:
                FP += 1

    # Calculate FN (missed positive segments)
    FN = len(ground_truth_df[ground_truth_df["label"] == 1]) - TP

    # Calculate precision, recall, and F1 score
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * (precision * recall) / (precision + recall)

    # Calculate AUC-ROC
    # auc_roc = roc_auc_score(ground_truth_df["label"], predictions_df["label"])

    return TP, FP, FN, precision, recall, f1

# Example usage:
# ground_truth_df and predictions_df are your actual dataframes
# Replace them with your actual data
ground_truth_df = PositiveCalls
predictions_df = metadata_pred

TP_val, FP_val, FN_val, precision_val, recall_val, f1_score_val = evaluate_segments(ground_truth_df, predictions_df)

print(f"True Positives: {TP_val}")
print(f"False Positives: {FP_val}")
print(f"False Negatives: {FN_val}")
print(f"Precision: {precision_val:.4f}")
print(f"Recall: {recall_val:.4f}")
print(f"F1 Score: {f1_score_val:.4f}")
# print(f"AUC-ROC: {auc_roc_val:.4f}")

### TODO: Calculate the actual begin_time and end_time by the wav file length (they are currently taken as absolute time to all the wav files as one recording instead of file by file)


True Positives: 65
False Positives: 86
False Negatives: 496
Precision: 0.4305
Recall: 0.1159
F1 Score: 0.1826


In [25]:
# ### SCRIPT TO FIX ABSOLUTE TIMES TO RELATIVE TIMES FOR EACH FILE ###

# import os
# import re
# import pandas as pd
# from datetime import datetime, timedelta
# import wave

# # Path to the directory containing the files
# annotation_path = r'C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\Placencia\verified_annotations'
# wav_path = r"C:\Users\amitg\Documents\Deep_Voice\HF_WAV_Manatee_Samples\Placencia\\"

# # Regex to extract base identifier from filenames
# regex = r'(\d+\.\d+)\.'

# # Gather all wav and txt files
# wav_files = sorted([f for f in os.listdir(wav_path) if f.endswith('.wav')])
# txt_files = sorted([f for f in os.listdir(annotation_path) if f.endswith('.txt') and 'selections' in f])

# # Ensure matched files between wav and txt
# matched_files = []
# for txt in txt_files:
#     match = re.search(regex, txt)
#     if match:
#         base_id = match.group(1)
#         wav = next((w for w in wav_files if base_id in w), None)
#         if wav:
#             matched_files.append((wav, txt))

# # Calculate cumulative duration of previous wav files
# cumulative_duration = 0
# previous_wav = None

# for wav, txt in matched_files:
#     if previous_wav:
#         with wave.open(os.path.join(wav_path, previous_wav), 'r') as wav_file:
#             frames = wav_file.getnframes()
#             rate = wav_file.getframerate()
#             duration = frames / float(rate)
#             cumulative_duration += duration

#     # Read the txt file into a DataFrame
#     txt_path = os.path.join(annotation_path, txt)
#     df = pd.read_csv(txt_path, sep='\t')
#     df.rename(columns={'Begin Time (s)': 'begin_time', 'End Time (s)': 'end_time'}, inplace=True) 
    
#     # Adjust begin and end times
#     df['begin_time'] -= cumulative_duration
#     df['end_time'] -= cumulative_duration

#     # Print the modified DataFrame (for debugging)
#     print(f"Adjusted content for {txt}:\n{df}")
#     new_txt_path = os.path.join(annotation_path, f"adjusted_{txt}")
#     df.to_csv(new_txt_path, sep='\t', index=False)
#     print(f"Saved adjusted content to {new_txt_path}")

#     # Update previous wav to current
#     previous_wav = wav

# print("Annotation times adjusted relative to individual files.")



Adjusted content for 67670025.201212101523.Table.1.selections_bb.txt:
    Selection           View  Channel    begin_time      end_time  \
0           1  Spectrogram 1        1    412.844775    413.186087   
1           2  Spectrogram 1        1    415.558280    415.731153   
2           3  Spectrogram 1        1    418.951454    419.321578   
3           4  Spectrogram 1        1    463.214077    463.542092   
4           5  Spectrogram 1        1    496.355213    496.571045   
..        ...            ...      ...           ...           ...   
59         68  Spectrogram 1        1  10011.874359  10012.568104   
60         69  Spectrogram 1        1  10027.661406  10028.316610   
61         70  Spectrogram 1        1  10045.524884  10046.164671   
62         71  Spectrogram 1        1  10081.875490  10081.956800   
63         72  Spectrogram 1        1  10102.389140  10102.456150   

    Low Freq (Hz)  Delta Time (s)  Delta Freq (Hz)  High Freq (Hz)  
0        9144.000          0.341